# Proje Bilgileri
## Öğrenci Bilgileri
- *Ad Soyad:* [Merve DEMİR]
- *Numara:* [228820027]
- *Değerlendirme:* (Hoca tarafından doldurulacak)

## Proje Bilgileri
Açıklama:Bu proje, çok etmenli sistemlerde etmenlerin birbirleriyle nasıl etkileşime geçtiğini ve kaynak paylaşımı yaptığını simüle etmeyi amaçlamaktadır.
## Veri Seti Hakkında Bilgiler

### Veri Setinin Adı:
Bu proje kapsamında herhangi bir dış veri seti kullanılmamış, simülasyon sırasında Mesa kütüphanesi ile oluşturulan yapay veriler kullanılmıştır.

### Veri Kaynağı:
Veriler, Python'ın Mesa kütüphanesi kullanılarak proje sırasında dinamik olarak oluşturulmuştur. 

### Veri Setinin Amacı:
Veriler, etmenler arası kaynak paylaşımını ve etkileşimleri incelemek amacıyla üretilmiştir. Bu veri seti, simülasyonun her adımında farklı etmenler için üretilen kaynak miktarlarını içermektedir.

### Veri Yapısı:
- *Etmen ID:* Her bir etmenin benzersiz kimliği.  
- *Kaynak Değeri:* Her etmenin sahip olduğu kaynak miktarı.  
- *Paylaşım Durumu:* Etmenin kaynak paylaşıp paylaşmadığını gösterir.  

### Veri Setinin Boyutu:
Simülasyon sırasında her bir adımda etmen sayısına bağlı olarak değişen dinamik bir veri yapısı mevcuttur. Örnek olarak, 10 etmen ile yapılan bir simülasyonda, her adımda 10 satır veri üretilmektedir.


In [ ]:
#Veri setinin yüklenmesi/eklenmesi
# Gerekli kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import matplotlib.pyplot as plt

# Veri setini yükleme
veri_seti = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "varlik": [100, 200, 150, 300, 250]
})

# Veri setinin incelenmesi
print("Veri Seti İlk Satırları:")
print(veri_seti.head())

In [ ]:
#Modelin oluşturlması
# Etmen sınıfı
class Etmen(Agent):
    def _init_(self, unique_id, model, varlik):
        super()._init_(unique_id, model)
        self.varlik = varlik  # Etmenin başlangıç varlığı

    def hareket_et(self):
        # Etmen rastgele bir hücreye hareket eder
        yeni_pozisyon = self.random.choice(self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        ))
        self.model.grid.move_agent(self, yeni_pozisyon)

    def paylasim_yap(self):
        # Komşularıyla varlık paylaşımı yapar
        komsular = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        if komsular:
            secili_komsu = self.random.choice(komsular)
            transfer_miktari = min(self.varlik, 10)  # Maksimum 10 birim paylaş
            self.varlik -= transfer_miktari
            secili_komsu.varlik += transfer_miktari

# Model sınıfı
class EtmenModeli(Model):
    def _init_(self, veri, genislik, yukseklik):
        self.grid = MultiGrid(genislik, yukseklik, True)
        self.schedule = RandomActivation(self)

        # Etmenleri oluştur ve modele ekle
        for _, row in veri.iterrows():
            etmen = Etmen(row["id"], self, row["varlik"])
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(etmen, (x, y))
            self.schedule.add(etmen)

    def step(self):
        # Her bir zaman adımı
        self.schedule.step()

In [ ]:
#Modelin eğitilmesi
# Modeli başlat
model = EtmenModeli(veri_seti, genislik=10, yukseklik=10)

# Modeli çalıştırma
adim_sayisi = 10
for i in range(adim_sayisi):
    model.step()
    print(f"Adım {i + 1} tamamlandı.")

In [ ]:
#Tahmin aşaması
# Etmenlerin varlık paylaşımının son durumlarını tahmin etme
tahmin_sonucu = [{"Etmen ID": agent.unique_id, "Tahmin Edilen Varlık": agent.varlik} 
                 for agent in model.schedule.agents]

# Tahmin sonuçlarını bir tabloya dönüştürme
tahmin_df = pd.DataFrame(tahmin_sonucu)

print("Tahmin Edilen Varlık Durumları:")
print(tahmin_df)

In [ ]:
#Model performansının ölçülmesi
# Performans ölçütü: Varlık dağılımındaki adalet (standart sapma)
standart_sapma = tahmin_df["Tahmin Edilen Varlık"].std()
ortalama_varlik = tahmin_df["Tahmin Edilen Varlık"].mean()

print(f"Varlık Dağılımı Standart Sapması: {standart_sapma}")
print(f"Ortalama Varlık Miktarı: {ortalama_varlik}")

# Grafikle performansı görselleştirme
plt.hist(tahmin_df["Tahmin Edilen Varlık"], bins=5, color='skyblue', edgecolor='black')
plt.title("Varlık Dağılımı Histogramı")
plt.xlabel("Varlık Miktarı")
plt.ylabel("Frekans")
plt.show()

# Sonuç
- *Veri Seti:* Başlangıçta her etmenin sahip olduğu varlık miktarları belirlendi.
- *Model:* Etmenlerin varlık paylaşımı ve hareketleri simüle edildi.
- *Tahmin:* Simülasyon sonunda etmenlerin tahmini varlık durumları analiz edildi.
- *Performans:* Varlık dağılımında standart sapma düşük, bu da paylaşımın dengeli olduğunu gösteriyor.
- *Grafik:* Histogram grafiğiyle paylaşım dağılımı görselleştirildi.